# Reassign DecisionTree leaves by $\Delta$IA

## Compute SNR

In [ ]:
import numpy as np
import pickle

In [ ]:
leaf_df = {}
for i in range(4):
    file = f'/global/cfs/cdirs/des/elisa/ShearSplits_data/decisiontree/leaf-label_deltas_bin{i}.pkl'
    leaf_df[i] = pickle.load(open(file, 'rb'), encoding='latin1')

In [ ]:
def SNR(signal, cov):
    return np.sqrt(np.dot(np.dot(signal.T, np.linalg.inv(cov)), signal))

In [ ]:
for i in range(4):
    print(f'Bin{i}')

    output_dir = f'/global/cfs/cdirs/des/elisa/ShearSplits_data/decisiontree/cosmosis_runs/bin{i}/full'

    deltaIA = np.loadtxt(f'{output_dir}/shear_planck/delta_IA.txt')
    cov_deltaIA = np.load(f'{output_dir}/covariance_delta_IA.npy')
    cov_cosmo = np.load(f'{output_dir}/covariance_cosmo.npy')
    cov = cov_deltaIA + cov_cosmo
    
    leaf_df_i = leaf_df[i]
    leaf_df_i['label_final'] = np.copy(leaf_df_i['label'])
    unique_leaf_ids = leaf_df_i['leaf_id'].unique()

    SNR_original = SNR(deltaIA, cov)
    print('SNR =', SNR_original)
    
    SRN_updated = SNR_original

    for leaf_id in unique_leaf_ids:
        if leaf_id % 1000 == 0: print(f'   leaf {leaf_id}')

        mask = leaf_df_i['leaf_id'] == leaf_id
        DdeltaIA = leaf_df_i.loc[mask, 'DdeltaIA'].values[0]
        Dcov_deltaIA = leaf_df_i.loc[mask, 'Dcov_deltaIA'].values[0]
        Dcov_cosmo = leaf_df_i.loc[mask, 'Dcov_cosmo'].values[0]

        SRN_new = SNR(deltaIA + DdeltaIA, (cov_deltaIA + Dcov_deltaIA) + (cov_cosmo + Dcov_cosmo))

        if SRN_new > SRN_updated:
            print('Changing leaf assignment: leaf_id =', leaf_id, ', SNR =', SRN_new)
            leaf_df_i.loc[mask, 'label_final'] = 1 - leaf_df_i.loc[mask, 'label_final'] # Change final assignment
            SRN_updated = SRN_new

    cols_to_save = ['leaf_id', 'label', 'DdeltaIA', 'Dcov_deltaIA', 'Dcov_cosmo', 'label_final']
    leaf_df_i[cols_to_save].to_pickle(f'/global/cfs/cdirs/des/elisa/ShearSplits_data/decisiontree/leaf-label_deltas_newlabels_bin{i}.pkl')

In [ ]:
# For a second round, take into account those leaves that have been changed and use -DdeltaIA, -Dcov_deltaIA -Dcov_cosmo ?